# NOTEBOOK PARA PLOTAGEM DA REGIÃO DO PIXEL DE EXTRAÇÃO DA INFORMAÇÃO

In [1]:
# manipulação de dados
import json
import numpy as np
import pandas as pd

# visualização geoespacial
import folium
from folium import Choropleth, Circle, Marker, Icon, Popup
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import Image, display

# plotagem (gráficos)
import seaborn as sns
import matplotlib.pyplot as plt

#análise geospacial
import ee
import collections
import geopandas as gpd
import geemap
import geemap.colormaps as colormaps

#manipulação de datas
from datetime import timedelta
from datetime import datetime

#manipulação de pastas do windows
import os

#estatística
import statistics
from statistics import mean
import scipy
from scipy.optimize import curve_fit
from scipy.stats import spearmanr
from scipy.stats import zscore

#regressões, métricas e modelos
from sklearn.model_selection import train_test_split #split database in training and testing data
from sklearn.metrics import * #accuracy_metrics
from sklearn.ensemble import RandomForestRegressor #randomforest regression
from sklearn.linear_model import LinearRegression #linear regression or multiple linear regression
from sklearn.neural_network import MLPRegressor #neural network
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR #svm regression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [2]:
import collections
collections.Callable = collections.abc.Callable

In [3]:
#ee.Authenticate()
ee.Initialize()

In [4]:
def maskl8clouds(image):
    '''
    Create mask for cloudy pixels and scale pixels to 
    respective reflectance (%) values for Landsat-8
    SR images.
    
    Parameters
    -----------
    image: ee.image.Image
        Google Earth Engine Image

        
    Returns
    -----------
    image: ee.image.Image
        Google Earth Engine Image
    
    '''
    
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    
    
    qa_mask = image.select("QA_PIXEL").bitwise_and(cloudShadowBitMask).eq(0)
    saturation_mask = image.select("QA_PIXEL").bitwise_and(cloudsBitMask).eq(0)
    
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    
    return image.addBands(opticalBands, None, True) \
                .addBands(thermalBands, None, True) \
                .updateMask(qa_mask) \
                .updateMask(saturation_mask) \
                .copyProperties(image, ['LANDSAT_PRODUCT_ID', 'SPACECRAFT_ID', 'system:time_start'])

In [5]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [6]:
def createBuffer(feature, inner_size):
    '''
    Create squared (area) buffer around feature
    
    Parameters:
    -----------
    feature: ee.Feature,
        Google Earth Engine feature
    inner_size: int
        area to create a buffer on

    Returns
    -----------
    inner: ee.Feature 
        Google Earth Engine buffered feature
    '''
    inner = feature.buffer(ee.Number(inner_size).sqrt().divide(2), 1).bounds()
    return inner

In [7]:
df = pd.read_pickle("df_matchups_l8_prov").reset_index()
df.columns

Index(['index', 'img_id', 'satellite', 'Site', 'Latitude (decimal deg)',
       'Longitude (decimal deg)', 'date', 'timestamp', 'l8_blue', 'l8_green',
       'l8_red', 'l8_nir', 'l8_swir1', 'l8_swir2', 'l8_temp', 'l8_aerosol',
       'l8_pixel_qa', 'Date', 'Station Depth (m)', 'Sample Depth (m)',
       'Sample Depth (category)', 'Local Time (Eastern Time Zone)',
       'Wind speed (knots)', 'Wave Height (ft)', 'Sky', 'SDD (m)',
       'Turbidity (NTU)', 'Chla (µg/L)', 'CDOM 400 nm (m-1)', 'TSS (mg/L)',
       'geometry'],
      dtype='object')

In [8]:
for idx, row in df.iterrows():
    img_id = row['img_id']
    lat, lon = row['Latitude (decimal deg)'], row['Longitude (decimal deg)']
    bounds = ee.Geometry.Point([lon, lat])
    sq_bounds = createBuffer(bounds, (30*3)**2)
    s_date = str((row['date'] + timedelta(days=-1)).date()) 
    e_date = str((row['date'] + timedelta(days=1)).date()) 
    l8_col = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(sq_bounds).filterDate(s_date, e_date).map(maskl8clouds)
    image = l8_col.first()
    url = image.getThumbUrl({'min': 0, 'max': 0.25, 'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'region': sq_bounds, 'dimensions': 256})
    print(row)
    img = Image(url=url, embed=False, format='png')       
    display(img)

index                                                                   86
img_id                            LC08_L2SP_019031_20150930_20200908_02_T1
satellite                                                        LANDSAT_8
Site                                                                  WE14
Latitude (decimal deg)                                             41.5541
Longitude (decimal deg)                                           -83.0146
date                                                   2015-09-30 00:00:00
timestamp                                       2015-09-30 13:10:23.371000
l8_blue                                                           0.097869
l8_green                                                          0.206135
l8_red                                                            0.239032
l8_nir                                                            0.197155
l8_swir1                                                          0.172206
l8_swir2                 

index                                                                  641
img_id                            LC08_L2SP_020031_20170809_20200903_02_T1
satellite                                                        LANDSAT_8
Site                                                                  WE15
Latitude (decimal deg)                                             41.6158
Longitude (decimal deg)                                           -83.0119
date                                                   2017-08-09 00:00:00
timestamp                                       2017-08-09 13:16:34.464000
l8_blue                                                           0.006095
l8_green                                                          0.052892
l8_red                                                            0.011693
l8_nir                                                            0.007398
l8_swir1                                                          0.016328
l8_swir2                 

index                                                                  617
img_id                            LC08_L2SP_020031_20170606_20201015_02_T1
satellite                                                        LANDSAT_8
Site                                                                  WE15
Latitude (decimal deg)                                             41.6163
Longitude (decimal deg)                                           -83.0134
date                                                   2017-06-06 00:00:00
timestamp                                       2017-06-06 13:16:12.227000
l8_blue                                                           0.022206
l8_green                                                           0.09179
l8_red                                                            0.067281
l8_nir                                                            0.021759
l8_swir1                                                          0.042417
l8_swir2                 

index                                                                  582
img_id                            LC08_L2SP_020031_20160907_20200906_02_T1
satellite                                                        LANDSAT_8
Site                                                                  WE15
Latitude (decimal deg)                                             41.6167
Longitude (decimal deg)                                           -83.0109
date                                                   2016-09-07 00:00:00
timestamp                                       2016-09-07 13:16:46.166000
l8_blue                                                           0.036255
l8_green                                                          0.059946
l8_red                                                            0.021718
l8_nir                                                            0.006722
l8_swir1                                                          0.002042
l8_swir2                 

index                                                                  626
img_id                            LC08_L2SP_020031_20170708_20200903_02_T1
satellite                                                        LANDSAT_8
Site                                                                  WE15
Latitude (decimal deg)                                             41.6167
Longitude (decimal deg)                                           -83.0111
date                                                   2017-07-08 00:00:00
timestamp                                       2017-07-08 13:16:21.295000
l8_blue                                                           0.021642
l8_green                                                          0.054872
l8_red                                                             0.02345
l8_nir                                                            0.021933
l8_swir1                                                          0.038901
l8_swir2                 

...

index                                                                  371
img_id                            LC08_L2SP_020030_20190714_20200827_02_T1
satellite                                                        LANDSAT_8
Site                                                                   SB4
Latitude (decimal deg)                                             43.7373
Longitude (decimal deg)                                           -83.8765
date                                                   2019-07-14 00:00:00
timestamp                                       2019-07-14 13:16:06.101000
l8_blue                                                           0.014894
l8_green                                                          0.022217
l8_red                                                            0.010835
l8_nir                                                            0.001302
l8_swir1                                                          0.000163
l8_swir2                 

index                                                                 1068
img_id                            LC08_L2SP_021030_20160712_20200906_02_T1
satellite                                                        LANDSAT_8
Site                                                                   SB4
Latitude (decimal deg)                                             43.7374
Longitude (decimal deg)                                           -83.8761
date                                                   2016-07-12 00:00:00
timestamp                                       2016-07-12 13:22:16.484000
l8_blue                                                           0.007896
l8_green                                                          0.046869
l8_red                                                            0.027688
l8_nir                                                            0.026984
l8_swir1                                                           0.03787
l8_swir2                 

index                                                                 1148
img_id                            LC08_L2SP_021030_20180819_20200831_02_T1
satellite                                                        LANDSAT_8
Site                                                                   SB4
Latitude (decimal deg)                                             43.7374
Longitude (decimal deg)                                           -83.8769
date                                                   2018-08-19 00:00:00
timestamp                                       2018-08-19 13:21:48.378000
l8_blue                                                           0.040783
l8_green                                                          0.090584
l8_red                                                            0.048507
l8_nir                                                            0.022068
l8_swir1                                                          0.023322
l8_swir2                 

index                                                                  985
img_id                            LC08_L2SP_021029_20180819_20200831_02_T1
satellite                                                        LANDSAT_8
Site                                                                   SB4
Latitude (decimal deg)                                             43.7374
Longitude (decimal deg)                                           -83.8769
date                                                   2018-08-19 00:00:00
timestamp                                       2018-08-19 13:21:24.495000
l8_blue                                                           0.031873
l8_green                                                          0.086866
l8_red                                                            0.043754
l8_nir                                                            0.019779
l8_swir1                                                          0.023031
l8_swir2                 

index                                                                  287
img_id                            LC08_L2SP_020030_20170606_20200903_02_T1
satellite                                                        LANDSAT_8
Site                                                                   SB4
Latitude (decimal deg)                                             43.7374
Longitude (decimal deg)                                           -83.8769
date                                                   2017-06-06 00:00:00
timestamp                                       2017-06-06 13:15:48.336000
l8_blue                                                           0.039409
l8_green                                                          0.056713
l8_red                                                            0.037419
l8_nir                                                            0.021294
l8_swir1                                                           0.03889
l8_swir2                 